# Universidade Federal do Rio Grande do Norte
## Programa de Pós-graduação em Engenharia Elétrica e de Computação
### Course: Tópicos Especiais C
### Professor: Ivanovitch Silva
### Students: 
* Aguinaldo Bezerra Batista Júnior
* Pedro Klisley Ferreira da Silva
* Ycaro Ravel Dantas

#### Activity: Task 2, Project 1
#### Subject: Finding Patterns in Crime
#### Objectives: Perform data analysis of a given dataset following guidance and hints in the professor's notebook.
#### Dataset: MontgomeryCountyCrime2013.csv

### 1 Getting to know the proposed dataset

Loading the given dataset into Pandas object and showing some rows

In [69]:
import pandas as pd
crimes = pd.read_csv("MontgomeryCountyCrime2013.csv")
crimes.head()

,Incident ID,CR Number,Dispatch Date / Time,Class,Class Description,Police District Name,Block Address,City,State,Zip Code,...,Sector,Beat,PRA,Start Date / Time,End Date / Time,Latitude,Longitude,Police District Number,Location,Address Number
0,200939101,13047006,10/02/2013 07:52:41 PM,511,BURG FORCE-RES/NIGHT,OTHER,25700 MT RADNOR DR,DAMASCUS,MD,20872.0,...,NaN,NaN,NaN,10/02/2013 07:52:00 PM,NaN,NaN,NaN,OTHER,NaN,25700.0
1,200952042,13062965,12/31/2013 09:46:58 PM,1834,CDS-POSS MARIJUANA/HASHISH,GERMANTOWN,GUNNERS BRANCH RD,GERMANTOWN,MD,20874.0,...,M,5M1,470.0,12/31/2013 09:46:00 PM,NaN,NaN,NaN,5D,NaN,NaN
2,200926636,13031483,07/06/2013 09:06:24 AM,1412,VANDALISM-MOTOR VEHICLE,MONTGOMERY VILLAGE,OLDE TOWNE AVE,GAITHERSBURG,MD,20877.0,...,P,6P3,431.0,07/06/2013 09:06:00 AM,NaN,NaN,NaN,6D,NaN,NaN
3,200929538,13035288,07/28/2013 09:13:15 PM,2752,FUGITIVE FROM JUSTICE(OUT OF STATE),BETHESDA,BEACH DR,CHEVY CHASE,MD,20815.0,...,D,2D1,11.0,07/28/2013 09:13:00 PM,NaN,NaN,NaN,2D,NaN,NaN
4,200930689,13036876,08/06/2013 05:16:17 PM,2812,DRIVING UNDER THE INFLUENCE,BETHESDA,BEACH DR,SILVER SPRING,MD,20815.0,...,D,2D3,178.0,08/06/2013 05:16:00 PM,NaN,NaN,NaN,2D,NaN,NaN


Listing the columns of the dataset and their types

In [70]:
crimes.columns

Index(['Incident ID', 'CR Number', 'Dispatch Date / Time', 'Class',
       'Class Description', 'Police District Name', 'Block Address', 'City',
       'State', 'Zip Code', 'Agency', 'Place', 'Sector', 'Beat', 'PRA',
       'Start Date / Time', 'End Date / Time', 'Latitude', 'Longitude',
       'Police District Number', 'Location', 'Address Number'],
      dtype='object')

In [71]:
crimes.dtypes

Incident ID                 int64
CR Number                   int64
Dispatch Date / Time       object
Class                       int64
Class Description          object
Police District Name       object
Block Address              object
City                       object
State                      object
Zip Code                  float64
Agency                     object
Place                      object
Sector                     object
Beat                       object
PRA                       float64
Start Date / Time          object
End Date / Time            object
Latitude                  float64
Longitude                 float64
Police District Number     object
Location                   object
Address Number            float64
dtype: object

By inspecting column names and some cells, we figured out some relations between some columns. Columns **"Police Distric Name"** and **"Police District Number"** stand for the same information: the identification of the Police District (Name or Code). Similarily, columns **"Class"** and **"Class Description"** refer to crime classes codes and their descriptions. **"Location"** is clearly a join of the information on **"Latitude"** and **"Longitude" columns**. A quick google search allowed to perceive the meaning of remaining non self-explanatory columns. **"CR Number"** refers to the Police Report Number, **"Beat"** stands for a police patrol area (subset of a Sector) and **"PRA"**, the Police Response Area, is a subset of Beat.



The column **"Zip Code"**, as already pointed in the guidance notebook, contains wrong type data. It is important to convert wrong or inadequate type data to more appropriate ones to help further analysis.

We could convert data in the **"Zip Code"** and **"Address Number"** columns to the more suitable interger type. However, these columns are full of _NaN_ and the lack of _NaN_ and _NA_ representations in integer columns is a well known Pandas "gotcha". Maybe thats why these columns were built as float type data (compatible with NAs and NaNs). A workaround to this problem could be the definition of a sentinel interger values for this kind of data absense.

NaNs and NAs are likeky to happen in a dataset during data acquisition, transmission and conversion steps. 
They negatively affect the quality of the dataset and they should be carefuly spotted and evaluated, as they may
lead to weak or innacurate conclusions.

### 2 Preparing Data

Converting some columns to more appropriate data types (see if it is really necessary)

Converting Date/Time data in columns **"Dispatch Date / Time"**, **"Start Date / Time"** and **"End Date / Time"**
from a generic Pandas object to a more convinient Date/Time Pandas object. This will allow the further extraction of date and time components easily. The resulting converted columns are going to be added right next to the original data columns.

In [72]:
crimes.insert(3, "datetime_Dispatch", pd.to_datetime(crimes["Dispatch Date / Time"]))

In [73]:
crimes.insert(17, "datetime_Start", pd.to_datetime(crimes["Start Date / Time"]))

In [74]:
crimes.insert(19, "datetime_End", pd.to_datetime(crimes["End Date / Time"]))

In [75]:
crimes.head()

,Incident ID,CR Number,Dispatch Date / Time,datetime_Dispatch,Class,Class Description,Police District Name,Block Address,City,State,...,PRA,Start Date / Time,datetime_Start,End Date / Time,datetime_End,Latitude,Longitude,Police District Number,Location,Address Number
0,200939101,13047006,10/02/2013 07:52:41 PM,2013-10-02 19:52:41,511,BURG FORCE-RES/NIGHT,OTHER,25700 MT RADNOR DR,DAMASCUS,MD,...,NaN,10/02/2013 07:52:00 PM,2013-10-02 19:52:00,NaN,NaT,NaN,NaN,OTHER,NaN,25700.0
1,200952042,13062965,12/31/2013 09:46:58 PM,2013-12-31 21:46:58,1834,CDS-POSS MARIJUANA/HASHISH,GERMANTOWN,GUNNERS BRANCH RD,GERMANTOWN,MD,...,470.0,12/31/2013 09:46:00 PM,2013-12-31 21:46:00,NaN,NaT,NaN,NaN,5D,NaN,NaN
2,200926636,13031483,07/06/2013 09:06:24 AM,2013-07-06 09:06:24,1412,VANDALISM-MOTOR VEHICLE,MONTGOMERY VILLAGE,OLDE TOWNE AVE,GAITHERSBURG,MD,...,431.0,07/06/2013 09:06:00 AM,2013-07-06 09:06:00,NaN,NaT,NaN,NaN,6D,NaN,NaN
3,200929538,13035288,07/28/2013 09:13:15 PM,2013-07-28 21:13:15,2752,FUGITIVE FROM JUSTICE(OUT OF STATE),BETHESDA,BEACH DR,CHEVY CHASE,MD,...,11.0,07/28/2013 09:13:00 PM,2013-07-28 21:13:00,NaN,NaT,NaN,NaN,2D,NaN,NaN
4,200930689,13036876,08/06/2013 05:16:17 PM,2013-08-06 17:16:17,2812,DRIVING UNDER THE INFLUENCE,BETHESDA,BEACH DR,SILVER SPRING,MD,...,178.0,08/06/2013 05:16:00 PM,2013-08-06 17:16:00,NaN,NaT,NaN,NaN,2D,NaN,NaN


### 4 Analyzing locations of crimes

There are a few columns that encode information about the location of crimes:

- <span style="background-color: #F9EBEA; color:##C0392B">Police District Name</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Block Address</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Zip Code</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Sector</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Beat</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Latitude</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Longitude</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Police District Number</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Location</span>
- <span style="background-color: #F9EBEA; color:##C0392B">Address Number</span>

In [76]:
location_columns = ['Police District Name','Block Address','Zip Code','Sector','Beat',
                    'Latitude','Longitude','Police District Number','Location',
                    'Address Number']

These columns have varying numbers of missing values, and varying granularity. Some of the columns represent areas with large granularity (like police districts), whereas some represent areas with small granularity, like <span style="background-color: #F9EBEA; color:##C0392B">Latitude</span>, and <span style="background-color: #F9EBEA; color:##C0392B">Longitude</span>.


In order to decide which column to use to analyze the locations of crimes, you need to utilize the following criteria:

- **Granularity**. Areas that are too small aren't great, because only a few crimes were committed inside them, which makes it hard to analyze and compare. For example, if I tell you that Silver Spring (a city in Montgomery County) is the place with the most crimes, you'll know to avoid that area. However, if I tell you that a 100 foot section of Silver Spring has the most crimes, it won't be as helpful (it's unlikely that you'll ever be in that 100 foot section).

In [77]:
for column in location_columns:
    count = crimes[column].value_counts().count()
    print(column, ": ", count)

Police District Name :  8
Block Address :  8143
Zip Code :  49
Sector :  15
Beat :  42
Latitude :  8065
Longitude :  8065
Police District Number :  8
Location :  8065
Address Number :  286


In [78]:
#students_grades = crimes.groupby(["Police District Name"])["Beat"].value_counts()
#print(students_grades)

pol_dist_relation = crimes.groupby(["Police District Name"])["Police District Number"].value_counts()
print(pol_dist_count)

Police District Name  Police District Number
BETHESDA              2D                        3383
GERMANTOWN            5D                        2755
MONTGOMERY VILLAGE    6D                        3812
OTHER                 OTHER                        8
ROCKVILLE             1D                        3480
SILVER SPRING         3D                        5533
TAKOMA PARK           TPPD                        23
WHEATON               4D                        4375
Name: Police District Number, dtype: int64


We will check the granularity of the columns by the different values count of each column.
Limiting to variables with less than 50 different values, we will got these variables:

- Police District Name :  8  
- Zip Code :  49
- Sector :  15
- Beat :  42
- Police District Number :  8

From this analysis, the **Police District Name** and **Police District Number** are the variables with the best granularity for this problem, since both have few different values, but not too few.

- **Comprehensibility**. You looked up the Police District map of Montgomery County before, so it's simple to tell what area corresponds to district <span style="background-color: #F9EBEA; color:##C0392B">6</span>. However, what area does Beat <span style="background-color: #F9EBEA; color:##C0392B">5M1</span> correspond to? You may be able to look this up, but it's harder to comprehend.

In [79]:
for column in location_columns:
    result = crimes[column].value_counts().head().index
    print(column, ": ", result)

Police District Name :  Index(['SILVER SPRING', 'WHEATON', 'MONTGOMERY VILLAGE', 'ROCKVILLE',
       'BETHESDA'],
      dtype='object')
Block Address :  Index(['11100  VEIRS MILL RD', '700  RUSSELL AVE', '100  EDISON PARK DR',
       '7100  DEMOCRACY BLVD', '12000  CHERRY HILL RD'],
      dtype='object')
Zip Code :  Float64Index([20910.0, 20904.0, 20902.0, 20877.0, 20874.0], dtype='float64')
Sector :  Index(['A', 'G', 'I', 'P', 'D'], dtype='object')
Beat :  Index(['3G1', '3I1', '4L2', '1A3', '3I2'], dtype='object')
Latitude :  Float64Index([39.0373858785, 39.1506767386, 39.111781485, 39.0219752112,
              39.0526973055],
             dtype='float64')
Longitude :  Float64Index([-77.0516359828, -77.2040589532, -77.2352805179, -77.1466424589,
              -76.9531784238],
             dtype='float64')
Police District Number :  Index(['3D', '4D', '6D', '1D', '2D'], dtype='object')
Location :  Index(['(39.037385878480961, -77.051635982808946)',
       '(39.150676738601661, -77.20405

Observing the values of these columns, the **Police District Name** is the variable tha is most comprehensible, since we can know where the facts ocurred without any mapping.

- **Missing values**. If a column has a lot of missing values, that means that the conclusions you draw are less valid, because you don't know if the missing data is systematic (ie all data for a given district is missing) or random (equal amounts of data are missing from each district). You should try to select a column that has minimal missing values.

In [80]:
#Missing values
location_columns = ['Police District Name','Block Address','Zip Code','Sector','Beat',
                    'Latitude','Longitude','Police District Number','Location',
                    'Address Number']
crimes[location_columns].isnull().sum()

Police District Name        0
Block Address               0
Zip Code                   30
Sector                     46
Beat                        8
Latitude                  161
Longitude                 161
Police District Number      0
Location                  161
Address Number            132
dtype: int64

Observing the missing values of these columns, the **Police District Name**, **Block address**, **Beat** and **Police District Number** are the variables with less missing values.

Combining the granularity, comprehensibility and missing values criteria, we choose the variable **Police District Name** because it was best evaluated on these 3 aspects.

After choose the location variable to use, we could answer these questions:

- In what area did the most crimes occur? What physical locations (like cities) does this area correspond to?

In [81]:
print("Incident ID Null: ", crimes['Incident ID'].isnull().sum())

count = crimes.groupby(["Police District Name"])['Police District Number'].value_counts()
print (count.sort_values(ascending=False))

Incident ID Null:  0
Police District Name  Police District Number
SILVER SPRING         3D                        5533
WHEATON               4D                        4375
MONTGOMERY VILLAGE    6D                        3812
ROCKVILLE             1D                        3480
BETHESDA              2D                        3383
GERMANTOWN            5D                        2755
TAKOMA PARK           TPPD                        23
OTHER                 OTHER                        8
Name: Police District Number, dtype: int64


In [82]:
crimes.groupby(["Police District Name"])['City'].value_counts()

Police District Name  City              
BETHESDA              BETHESDA              1736
                      ROCKVILLE              534
                      CHEVY CHASE            496
                      KENSINGTON             275
                      POTOMAC                162
                      SILVER SPRING          158
                      CABIN JOHN              18
                      GLEN ECHO                4
GERMANTOWN            GERMANTOWN            2128
                      DAMASCUS               229
                      CLARKSBURG             173
                      GAITHERSBURG           119
                      BOYDS                   86
                      DICKERSON                7
                      BARNESVILLE              4
                      MOUNT AIRY               3
                      BROOKEVILLE              2
                      DERWOOD                  2
                      MONTGOMERY VILLAGE       1
                      POOLES

Silverspring is the area where the most crimes occur. It correspond to the area with the cities **SILVER SPRING, BURTONSVILLE, TAKOMA PARK, SPENCERVILLE, CHEVY CHASE, LAUREL**  

- Which area has the highest number of crimes per capita? You may be able to find population data per area online. For example, [this](https://www.montgomerycountymd.gov/POL/Resources/Files/crime/MCP2015AnnualCrimeReportFINAL.pdf) annual report has per-district populations towards the bottom.